# Widerstandsnetzwerke

In [1]:
# Bibliotheken importieren
import ipywidgets as widgets
from ipywidgets import Box, HBox, VBox, Layout, Image
from ipycanvas import MultiCanvas, Canvas, hold_canvas
from math import pi, sin, cos, atan, sqrt


# RANDOM COLOR FOR DEBUGGING

import random
def rand_color():
    random_number = random.randint(1048576,16777215)
    hex_number = str(hex(random_number))
    hex_number ='#'+ hex_number[2:]
    return hex_number

# Farben definieren
colors = {
    "black" : "#000000",
    "U"     : "#3333cc",
    "I"     : "#e8082c",
    "R"     : "#00d000",
    "lgrey" : "#f0f0f0",
    "dgrey" : "#c0c0c0",
    "diel"  : "#99ff99",
    "white" : "#ffffff",
    "elec"  : "#ffff00"
}

# Konstanten definieren

y0_top  =  50
y0_dist = 125

# Vorzeichnen redundanter Grafiken
el_dict = {}

# Widerstand
img_R = Canvas(width=40, height=18)
img_R.stroke_style = colors["black"]
img_R.line_width = 3
img_R.begin_path()
img_R.move_to( 2,  2)
img_R.line_to(38,  2)
img_R.line_to(38, 16)
img_R.line_to( 2, 16)
img_R.line_to( 2,  2)
img_R.line_to( 3,  2)
img_R.stroke()
img_R.fill_style = colors["white"]
img_R.fill()
el_dict["R"] = (img_R, 20, 9)

# Lampe
img_GL = Canvas(width=20, height=20)
# !!! später einfärben
img_GL.fill_style = "#ffffff" #colors["elec"]
img_GL.fill_arc(10, 10, 9, 0, 2 * pi)
#
img_GL.line_width = 1.25
img_GL.stroke_style = colors["black"]
img_GL.stroke_arc(10, 10, 9, 0, 2 * pi)
img_GL.stroke_style = colors["black"]
img_GL.begin_path()
img_GL.move_to( 4,  4)
img_GL.line_to(16, 16)
img_GL.stroke()
img_GL.begin_path()
img_GL.move_to(16,  4)
img_GL.line_to( 4, 16)
img_GL.stroke()
el_dict["GL"] = (img_GL, 10, 10)

# Knoten
img_node = Canvas(width=6, height=6)
img_node.fill_style = colors["black"]
img_node.fill_arc(3, 3, 3, 0, 2 * pi)
el_dict["node"] = (img_node, 3, 3)

# Schalter, offen, vertikal
img_vsw0 = Canvas(width=15, height=30)
img_vsw0.fill_style = "#ffffff"
img_vsw0.fill_rect(0, 0, 15, 30)
img_vsw0.draw_image(img_node, 9, 0)
img_vsw0.begin_path()
img_vsw0.move_to(13, 1)
img_vsw0.line_to(3, 27)
img_vsw0.stroke()
el_dict["Sv0"] = (img_vsw0, 12, 3)

# Schalter, offen, horizontal
img_hsw0 = Canvas(width=30, height=15)
img_hsw0.fill_style = "#ffffff"
img_hsw0.fill_rect(0, 0, 30, 15)
img_hsw0.draw_image(img_node, 0, 9)
img_hsw0.begin_path()
img_hsw0.move_to( 0, 13)
img_hsw0.line_to(27,  3)
img_hsw0.stroke()
el_dict["Sh0"] = (img_hsw0, 15, 12)

# Schalter, geschlossen, vertikal
img_vsw1 = Canvas(width=6, height=30)
img_vsw1.draw_image(img_node, 0, 3)
img_vsw1.begin_path()
img_vsw1.move_to(3,  4)
img_vsw1.line_to(3, 30)
img_vsw1.stroke()
el_dict["Sv1"] = (img_vsw1, 3, 3)

# Schalter, geschlossen, horizontal
img_hsw1 = Canvas(width=30, height=6)
img_hsw1.draw_image(img_node, 0, 0)
img_hsw1.begin_path()
img_hsw1.move_to(4,  3)
img_hsw1.line_to(30, 3)
img_hsw1.stroke()
el_dict["Sh1"] = (img_hsw1, 3, 3)

In [2]:
test = Canvas(width=160, height=20)
test.draw_image(img_node, 0, 7)
test.draw_image(img_node, 154, 7)
test.line_width = 1
test.begin_path()
test.move_to(  0, 10)
test.line_to(160, 10)
test.stroke()
test.draw_image(img_R, 40, 1)
test.draw_image(img_GL, 110, 0)

if 0:
    display(img_vsw0)
    display(img_vsw1)
    display(img_hsw0)
    display(img_hsw1)
    display(img_node)
    display(img_R)
    display(img_GL)#HBox([img_lamp, img_lamp, img_lamp, img_lamp, img_lamp, img_lamp, img_lamp]))
    display(test)

In [4]:
class GridElement():
    
    def __init__(self, parent, value="R", pointer=[None], backpointer=None):
        
        #self.etype  = etype
        self.parent = parent
        
        self.menu = widgets.Dropdown(options = [["-",                 "-"],
                                                ["Widerstand",        "R"],
                                                ["Lampe",             "GL"],
                                                ["Schalter",          "Sh0"],
                                                ["Reihenschaltung",   "ser"],
                                                ["Parallelschaltung", "par"]],
                                     layout  = Layout(width="145px"),
                                     value   = value)
        
        self.menu.observe(self.update, "value")
        
        self.pointer     = pointer
        self.backpointer = backpointer

    
    
    def update(self, *args):
                
        def f_pass(): pass
        
        def f_serial():
            self.menu.value = "R"
            self.pointer = [GridElement(parent      = self.parent,
                                        value       = self.menu.value,
                                        backpointer = self)]
            
        
        def f_parallel():
            self.menu.value = "R"
            
            self.backpointer.pointer.append(GridElement(parent      = self.parent,
                                                        value       = self.menu.value,
                                                        backpointer = self.backpointer))
        
        def f_remove():
            self.backpointer.pointer.remove(self)
            #self.pointer[0].backpointer = self.backpointer
        
        switcher = {
            "ser"    : f_serial,
            "par"    : f_parallel,
            "-"      : f_remove
        }
        
        switch_func = switcher.get(self.menu.value, f_pass)
        switch_func()
        
        self.parent.reconstruct()
        
    
    
    def draw_arrow(self, layer, xy, angle, length, color="#000000", line_width=2, tip_width=10, hold=True):
        
        sin_a = sin((angle + 90) * pi / 180)
        cos_a = cos((angle + 90) * pi / 180)

        if length < 15:
            tip_length = 15 + (length - 15)
            width = tip_width / 2 * length / 15
        else:
            tip_length = 15
            width = tip_width / 2
        line_end = (xy[0] + (length - tip_length) * sin_a,
                    xy[1] + (length - tip_length) * cos_a)        
        
        def do_draw(layer):
            
            # Pfeilschaft
            layer.line_width = line_width
            layer.stroke_style = color
            layer.fill_style   = color
            layer.begin_path()
            layer.move_to(xy[0], xy[1])
            layer.line_to(line_end[0], line_end[1])
            layer.stroke()
            layer.close_path()
            
            # Pfeilspitze
            layer.move_to(xy[0] + length * sin_a,
                          xy[1] + length * cos_a)
            layer.line_to(line_end[0] + width * sin((angle + 180) * pi / 180),
                          line_end[1] + width * cos((angle + 180) * pi / 180))
            layer.line_to(line_end[0] - width * sin((angle + 180) * pi / 180),
                          line_end[1] - width * cos((angle + 180) * pi / 180))
            layer.fill()
        
        if hold:
            with hold_canvas(layer):
                do_draw(layer)
        else:
            do_draw(layer)
    
    
    
    def draw(self, comp, grid, x, y):
        
        # x, y in Gitterkoordinaten
        
        #print(x, y)
        if self.menu:
            el = el_dict[self.menu.value]
        else:
            el = el_dict["Sv0"]
        
        x_dist   = 162.5
        x0_dist  = 106.25
        x_offset = el[1]
        
        y_dist   = 50
        y_offset = el[2]
        
        grid.begin_path()
        
        ###grid.stroke_style = "#000000"

        if x != -1:
            grid.move_to(    x * x_dist + 25, y * y_dist + y0_dist)
            # Horizontale
            grid.line_to((x+1) * x_dist + 25, y * y_dist + y0_dist)
        else:
            grid.move_to((x+1) * x_dist + 25, y * y_dist + y0_dist)
            
        
        
        """if x == -1:
            #x = 0
            grid.stroke_style = "#ff0000"
        else:
            grid.stroke_style = "#000000"
        """
        
        # Kein nachgeschaltetes Element
        if self.pointer == [None]:
            grid.line_to(675, y     * y_dist + y0_dist)
            if self.y > 0:
                grid.line_to(675, (y-1) * y_dist + y0_dist)
                grid.draw_image(img_node, 675 - 3, (y-1) * y_dist + y0_dist - 3)
        
        # Mehrere nachgeschaltete Elemente
        elif len(self.pointer) > 1:
            y_max = 0
            for pi in range(len(self.pointer)):
                p = self.pointer[pi]
                if p != self.pointer[-1]:
                    grid.draw_image(img_node, (x+1) * x_dist + 25 - 3,     p.y * y_dist + y0_dist - 3)
                if p.y > y_max: y_max = p.y
            # Vertikale
            grid.line_to(             (x+1) * x_dist + 25,     y_max * y_dist + y0_dist)
        
        grid.stroke()
        
        comp.draw_image(el[0], x*x_dist + x0_dist - x_offset, y*y_dist + y0_dist - y_offset)
        
        # Q'n'D
        comp.font = '14px serif'
        comp.text_align = "left"
        comp.fill_style = colors["I"]
        pre = "I"
        if self.menu:
            pre += self.menu.value
            if pre[1] != "S":
                comp.fill_style = colors["I"]
                self.draw_arrow(comp, (x*x_dist + x0_dist - 65,  y*y_dist + y0_dist), 0, 40, colors["I"], line_width=3, tip_width=10, hold=False)
            else:
                pre = pre[1]
                comp.fill_style = colors["black"]
            comp.fill_text(pre + str(self.i), x*x_dist + x0_dist - 65, y*y_dist + y0_dist - 15)
                
    
    
class Grid(GridElement):
    
    def __init__(self):
        
        layers = ["bg",
                  "grid",
                  "components"]
        lx = 700
        ly = 450
        
        self.canvas = MultiCanvas(len(layers), width=lx, height=ly)
        self.c = {}
        for li in range(len(layers)):
            self.c[layers[li]] = self.canvas[li]
        
        self.menu    = None #GridElement(value="hsw0", parent=self, pointer=[None], backpointer=self)
        self.pointer = [GridElement(value="R", parent=self, pointer=[None], backpointer=self)]
                
        self.out = widgets.Output()
        
        self.draw_background()
        self.reconstruct()
        
        display(self.canvas)
        display(self.out)
        
        #self.
    
    
    
    def draw_background(self):
        
        bg = self.c["bg"]
        
        bg.begin_path()
        bg.move_to( 25, y0_dist)
        bg.line_to( 25, y0_top)
        bg.line_to(675, y0_top)
        bg.line_to(675, y0_dist)
        bg.stroke()
        bg.line_width = 1.25
        bg.stroke_arc(350, 50, 20, 0, 2 * pi)
        
        bg.font = '14px serif'
        bg.text_align = "left"
        bg.fill_style = colors["U"]
        self.draw_arrow(bg, (322.5, 15), 0, 60, colors["U"], line_width=3, tip_width=10, hold=False)
        bg.fill_text("U0", 290, 20)
    
    
    
    def reconstruct(self):
        
        # --- Indizierung --- #
        
        #print()
        
        matrix = {}
        
        grid = self.c["grid"]
        comp = self.c["components"]
        
        self.el_counter = {}
        
        def rec(el, x=-1, y=0, R=0):
            
            if 1:#el != self:
                
                matrix[y, x] = el
                el.x, el.y = x, y
                
                if el.menu:
                    if el.menu.value in self.el_counter:
                        self.el_counter[el.menu.value] += 1
                    else:
                        self.el_counter[el.menu.value]  = 1
                    el.i = self.el_counter[el.menu.value]
                else:
                    el.i = ""
            
            if el.pointer != [None]:
                
                ys = []
                
                R = 0
                
                for i in range(len(el.pointer)):
                    
                    # Positionsbestimmung
                    if len(el.pointer) <= 1 or i == 0:
                        yd = y
                    else:
                        yd = y + 1
                        
                    rec(el.pointer[i], x=x+1, y=yd, R=1)

                    ys.append(y)
                    
                return max(ys), R
            
            else: return y, R
            
        response = rec(self)
        
        #print(response[1])
       
        index = list(matrix)
        index.sort()
        
        # --- Darstellung --- #
        
        x_last = -1
        y_last =  0
            
        rows = []
        row  = []
        
        grid.clear()
        comp.clear()
        
        # Maximalwerte finden
        
        x_max = 0
        
        for i in index:
            if i[1] > x_max: x_max = i[1]
                
        y_max = index[-1][0]
        
        # Elemente darstellen
        
        for i in index:
            
            x = i[1]
            y = i[0]
            
            if y != y_last:
                rows.append(HBox(row))
                row    = []
                x_last = -1

            # Mindestabstand (1,5m)
            if (x - x_last) > 1:
                for j in range(x - x_last - 1):
                    row.append(Box(layout=Layout(width="149px")))

            if x > -1:
                row.append(matrix[i].menu)
            
            with hold_canvas(grid):
                matrix[i].draw(comp, grid, x, y)
            
            x_last = x
            y_last = y
                
        rows.append(HBox(row))
        
        with self.out:
            self.out.clear_output()
            display(VBox(rows))



G = Grid()

MultiCanvas(height=450)

Output()